In [1]:
import pickle
import torch
from torch_geometric.data import Data,DataLoader
from functions_refactor import *
from pytorch_util import *
from torch.optim import Adam
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [2]:
# model parameters
reuse = False
block = MEGNet_block
head = feedforwardHead_Update
data = '../Data/{}_data_ACSF_expand_PCA.pickle'
batch_size = 32
dim = 128
epochs = 50
clip = 0.4
layer1 = 3
layer2 = 3
factor = 2
lr = 1e-4

In [3]:
prefix = '_MEGNet_5folds.csv'

In [4]:
train = pd.read_csv('../Data/train.csv')
test = pd.read_csv('../Data/test.csv')

In [5]:
folds = []
for f in range(5):
    with open(data.format('train').split('pickle')[0][:-1]+'_f'+str(f)+'.pickle', 'rb') as handle:
        folds.append(pickle.load(handle))
folds = [[Data(**d) for d in fold] for fold in folds]

In [ ]:
for i in range(5):
    print('\nstart fold '+str(i))
    # parpare data
    train_list = []
    val_list = []
    for j in range(5):
        if i == j:
            val_list.extend(folds[j])
        else:
            train_list.extend(folds[j])
    
    train_dl = DataLoader(train_list,batch_size,shuffle=True)
    val_dl = DataLoader(val_list,batch_size,shuffle=False)
    
    # train model
    model = GNN_edgeUpdate(reuse,block,head,dim,layer1,layer2,factor,**data_dict[data]).to('cuda:0')
    paras = trainable_parameter(model)
    opt = Adam(paras,lr=lr)
    scheduler = ReduceLROnPlateau(opt, 'min',factor=0.5,patience=5)
    model,train_loss_list,val_loss_list,bestWeight = train_type(opt,model,epochs,train_dl,val_dl,paras,clip,scheduler=scheduler)
    
    # predict oof for each type
    for type_i in range(8):
        # load val data and type_id
        with open(data.format('train').split('pickle')[0][:-1]+'_f'+str(i)+'_type_'+str(type_i)+'.pickle', 'rb') as handle:
            test_data = pickle.load(handle)
        test_list = [Data(**d) for d in test_data]
        test_dl = DataLoader(test_list,batch_size,shuffle=False)
        
        with open(data.format('train').split('pickle')[0][:-1]+'_f'+str(i)+'_type_'+str(type_i)+'_id.pickle', 'rb') as handle:
            test_id = pickle.load(handle)
    
        # load model
        model.load_state_dict(bestWeight[type_i])
    
        # predict
        model.eval()
        yhat_list = []
        with torch.no_grad():
            for data_torch in test_dl:
                data_torch = data_torch.to('cuda:0')
                yhat_list.append(model(data_torch,False,True))
        yhat = torch.cat(yhat_list).cpu().detach().numpy()        
    
        # join
        submit_ = dict(zip(test_id,yhat))
        train['fold'+str(i)+'_type'+str(type_i)] = train.id.map(submit_)
    
    # predict test
    for type_i in range(8):
        # load val data and type_id
        with open(data.format('test').split('pickle')[0][:-1]+'_type_'+str(type_i)+'.pickle', 'rb') as handle:
            test_data = pickle.load(handle)
        test_list = [Data(**d) for d in test_data]
        test_dl = DataLoader(test_list,batch_size,shuffle=False)
        
        with open(data.format('test').split('pickle')[0][:-1]+'_id_type_'+str(type_i)+'.pickle', 'rb') as handle:
            test_id = pickle.load(handle)
    
        # load model
        model.load_state_dict(bestWeight[type_i])
    
        # predict
        model.eval()
        yhat_list = []
        with torch.no_grad():
            for data_torch in test_dl:
                data_torch = data_torch.to('cuda:0')
                yhat_list.append(model(data_torch,False,True))
        yhat = torch.cat(yhat_list).cpu().detach().numpy()        
    
        # join
        submit_ = dict(zip(test_id,yhat))
        test['fold'+str(i)+'_type'+str(type_i)] = test.id.map(submit_)


start fold 0
epoch:0, train_loss: +0.358, val_loss: -0.051, 
train_vector: +4.22|+0.96|+0.12|-0.42|-0.57|+0.11|-0.61|-0.93, 
val_vector  : +3.92|+0.45|-0.28|-1.01|-1.10|-0.20|-1.03|-1.16

epoch:1, train_loss: -0.298, val_loss: -0.715, 
train_vector: +2.25|+0.34|-0.33|-0.96|-1.07|-0.25|-1.05|-1.31, 
val_vector  : +0.77|-0.22|-0.52|-1.43|-1.23|-0.37|-1.27|-1.45

epoch:2, train_loss: -0.629, val_loss: -0.569, 
train_vector: +0.89|+0.12|-0.51|-1.18|-1.27|-0.38|-1.21|-1.50, 
val_vector  : +0.60|-0.13|-0.59|-0.82|-0.84|-0.44|-0.88|-1.44

epoch:3, train_loss: -0.728, val_loss: -0.853, 
train_vector: +0.81|-0.04|-0.60|-1.31|-1.33|-0.46|-1.30|-1.60, 
val_vector  : +0.56|-0.38|-0.68|-1.30|-1.54|-0.54|-1.29|-1.66

epoch:4, train_loss: -0.825, val_loss: -0.949, 
train_vector: +0.77|-0.15|-0.69|-1.42|-1.49|-0.53|-1.38|-1.70, 
val_vector  : +0.17|+0.01|-0.75|-1.60|-1.62|-0.59|-1.44|-1.77

epoch:5, train_loss: -0.893, val_loss: -0.926, 
train_vector: +0.71|-0.23|-0.76|-1.50|-1.56|-0.59|-1.44|-1.77, 

In [ ]:
assert set(test.iloc[:,5:].isnull().sum(1)) == set([7*5])
test['yhat'] = np.nanmean(test.iloc[:,5:],1)
test = test[['id','yhat']]
test.to_csv('../Data/test'+prefix,index=False)

assert set(train.iloc[:,6:].isnull().sum(1)) == set([train.iloc[:,6:].shape[1]-1])
train['yhat'] = np.nanmean(train.iloc[:,6:],1)
train = train[['id','yhat']]
train.to_csv('../Data/train'+prefix,index=False)